<font color="#CC3D3D"><p>
# [Competition] Building a `Linear Model` with `Pipeline+Optuna`

<font color="blue"><p>
#### LM 모형 구축절차
1. 수치형 피처
 - 결측값처리: SimpleImputer(strategy=`???`)
 - 이상값처리: FunctionTransformer((remove_outlier, kw_args={'q':`???`})))
 - 스케일링: PowerTransformer()
2. 범주형 피처
 - 결측값처리: SimpleImputer(strategy="most_frequent")
 - 인코딩: OneHotEncoder(handle_unknown="ignore", sparse=True)
3. 공통
 - Feature Selection: SelectPercentile(percentile=`???`)
 - Modeling: Ridge(alpha=`???`)
 - Hyperparametor Optimization: `OptunaSearchCV`
 - OOF Prediction   

In [1]:
LM_VERSION = 1.0

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from category_encoders import TargetEncoder, BinaryEncoder
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import optuna
from optuna.distributions import CategoricalDistribution, IntDistribution, FloatDistribution
from optuna.integration import OptunaSearchCV, ShapleyImportanceEvaluator

#### Load data

In [3]:
X_train = pd.read_csv('X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

#### 수치형/범주형 피처 분리 & 학습/평가 데이터 분할

In [7]:
numeric_features = ['대학성적']
categorical_features = ['직종','세부직종','직무태그','근무경력','근무형태','근무지역','출신대학','대학전공','어학시험','자격증']

X_train = X_train[numeric_features+categorical_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features]

####  파이프라인 구축

In [8]:
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})),
        ("scaler", PowerTransformer()),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse=True)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
        ("selector", SelectPercentile(percentile=100)),
    ]
)

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("classifier", Ridge(alpha=1.0)),  #작으면 작을수록 과적합이 커진다. 적정값은 튜닝을 통해 알아내야한다.
    ]
)

set_config(display="diagram")  # To view the text pipeline, change to display='text'.
model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('column',
                                  ColumnTransformer(transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('outlier',
                                                                                    FunctionTransformer(func=<function remove_outlier at 0x000002F5E77471F0>,
                                                                                                        kw_args={'q': 0.05})),
                                                                                   ('scaler',
                                                                                    PowerTransformer())]),
                                                                   ['대학성적']),
                                                                  ('cat',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='most_frequent')),
                                                                                   ('encoder',
                                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                                   ['직종',
                                                                    '세부직종',
                                                                    '직무태그',
                                                                    '근무경력',
                                                                    '근무형태',
                                                                    '근무지역',
                                                                    '출신대학',
                                                                    '대학전공',
                                                                    '어학시험',
                                                                    '자격증'])])),
                                 ('selector',
                                  SelectPercentile(percentile=100))])),
                ('classifier', Ridge())])

#### LM Baseline 성능 확인

In [9]:
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5) #사이킷런 rmse 제공 x -> mse를 돌리고 neg_으로 -1을 곱해준다 

print("Default LM CV scores: ", np.sqrt(-1*scores))
print("Default LM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))

#데이터가 작으면 cv때 편차가 크면 리더보드에 쐈을때 이상한 결과가 나올수있다

Default LM CV scores:  [ 839.62243134  808.81131592  905.52429483  945.10484123 1101.38882215]
Default LM CV mean = 925.79 with std = 443.66


#### `파이프라인+Optuna`를 통한 LM 하이퍼파라미터 최적화

In [10]:
%%time

param_distributions = {
    "preprocessor__column__num__imputer__strategy": CategoricalDistribution(["mean","median"]),
    "preprocessor__column__num__outlier__kw_args": CategoricalDistribution([{'q':0.01},{'q':0.05},{'q':0.1}]),  
    "preprocessor__selector__percentile": IntDistribution(50,100,step=10), 
    "classifier__alpha": IntDistribution(1,10),
}

optuna.logging.set_verbosity(optuna.logging.WARNING)
optuna_search = OptunaSearchCV(model, param_distributions, cv=5, 
                               scoring='neg_mean_squared_error', n_trials=20,
                               study=optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction='maximize'))
optuna_search.fit(X_train, y_train)

CPU times: total: 1min 12s
Wall time: 12.2 s


OptunaSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                          Pipeline(steps=[('column',
                                                           ColumnTransformer(transformers=[('num',
                                                                                            Pipeline(steps=[('imputer',
                                                                                                             SimpleImputer(strategy='median')),
                                                                                                            ('outlier',
                                                                                                             FunctionTransformer(func=<function remove_outlier at 0x000002F5E77471F0>,
                                                                                                                                 kw_args={'q': 0.05})),
                                                                                                            ('scaler',
                                                                                                             PowerTransformer())]),
                                                                                            ['대학성적']),
                                                                                           ('cat',
                                                                                            Pipeline(steps=[...
                                    'preprocessor__column__num__imputer__strategy': CategoricalDistribution(choices=('mean', 'median')),
                                    'preprocessor__column__num__outlier__kw_args': CategoricalDistribution(choices=({'q': 0.01}, {'q': 0.05}, {'q': 0.1})),
                                    'preprocessor__selector__percentile': IntDistribution(high=100, log=False, low=50, step=10)},
               scoring='neg_mean_squared_error',
               study=<optuna.study.study.Study object at 0x000002F5E775B400>)

In [11]:
print(f"\nBest params: {optuna_search.best_params_}")
print(f"\nBest score: {np.sqrt(-1*optuna_search.best_score_):.2f}")


Best params: {'preprocessor__column__num__imputer__strategy': 'median', 'preprocessor__column__num__outlier__kw_args': {'q': 0.01}, 'preprocessor__selector__percentile': 90, 'classifier__alpha': 4}

Best score: 915.65


#### Submission 생성

In [12]:
# 옵튜나는 다시 모델링을 해야한다.
# 최적화된 하이퍼파라미터로 파이프라인 재설정

model.set_params(**optuna_search.best_params_)

# OOF Prediction
models = cross_validate(model, 
                        X_train, y_train, 
                        cv=5, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

#5번의 cv를 통해서 얻은 값들을 평균냄

scores = models['test_score']
print("\nTuned LM CV scores: ", np.sqrt(-1*scores))
print("Tuned LM CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))


Tuned LM CV scores:  [ 819.9454163   792.65524928  890.39230139  942.4883535  1100.17731068]
Tuned LM CV mean = 915.65 with std = 455.20


In [13]:
# submission 화일 생성
filename = f'LM_{LM_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)

<font color="#CC3D3D"><p>
# End